<a href="https://colab.research.google.com/github/javier-jaime/Route-Recommender/blob/main/Mexico_Municipal_Crime_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import Libraries
# !pip install folium
import pandas as pd
from geopy.geocoders import Nominatim
from google.colab import drive
from google.colab import files
import folium # map rendering library
from folium import plugins

In [3]:
# Mount Google Drive

drive.mount('/content/drive')

Mounted at /content/drive



Original Dataset from Mexico Government here:

https://www.gob.mx/sesnsp/acciones-y-programas/incidencia-delictiva-del-fuero-comun-nueva-metodologia

In [4]:
# Read dataset from excel in Google Drive to Pandas Data Frame
muni_crime_df = pd.read_excel('drive/MyDrive/Colab_Data/Municipal_2022.xlsx')
print(muni_crime_df.shape)
muni_crime_df.head()

(242942, 21)


,Año,Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,...,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2022,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2,...,2,0,0,3,0,NaN,NaN,NaN,NaN,NaN
1,2022,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,2,...,1,0,0,2,0,NaN,NaN,NaN,NaN,NaN
2,2022,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0,...,0,0,2,0,0,NaN,NaN,NaN,NaN,NaN
3,2022,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,2022,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Drop Empty/NA cells
muni_crime_df = muni_crime_df.dropna(axis=1)
muni_crime_df.isnull().any()

Año                       False
Clave_Ent                 False
Entidad                   False
Cve. Municipio            False
Municipio                 False
Bien jurídico afectado    False
Tipo de delito            False
Subtipo de delito         False
Modalidad                 False
Enero                     False
Febrero                   False
Marzo                     False
Abril                     False
Mayo                      False
Junio                     False
Julio                     False
dtype: bool

In [6]:
# Count of Unique Crime types
muni_crime_df['Tipo de delito'].value_counts()

Robo                                                                             89244
Homicidio                                                                        22311
Lesiones                                                                         22311
Secuestro                                                                        12395
Feminicidio                                                                       9916
Narcomenudeo                                                                      2479
Violencia de género en todas sus modalidades distinta a la violencia familiar     2479
Incumplimiento de obligaciones de asistencia familiar                             2479
Otros delitos contra la familia                                                   2479
Corrupción de menores                                                             2479
Trata de personas                                                                 2479
Otros delitos contra la sociedad           

In [7]:
# Count Unique Crime sub-types
muni_crime_df['Subtipo de delito'].value_counts()

Robo de maquinaria                                                               14874
Robo de vehículo automotor                                                       14874
Lesiones culposas                                                                12395
Secuestro                                                                        12395
Homicidio culposo                                                                12395
Homicidio doloso                                                                  9916
Lesiones dolosas                                                                  9916
Feminicidio                                                                       9916
Robo en transporte público individual                                             4958
Robo a transportista                                                              4958
Robo a transeúnte en vía pública                                                  4958
Robo a transeúnte en espacio abierto al púb

In [8]:
muni_crime_df['2022 AVG'] = (muni_crime_df['Febrero'] + muni_crime_df['Marzo'] + muni_crime_df['Abril'] + muni_crime_df['Mayo'] + muni_crime_df['Junio'] + muni_crime_df['Julio']) / 6
muni_crime_df.drop(['Año','Clave_Ent', 'Cve. Municipio','Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio'], axis=1, inplace=True)
muni_crime_df.head()

,Entidad,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,2022 AVG
0,Aguascalientes,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,1.000000
1,Aguascalientes,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,0.833333
2,Aguascalientes,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0.333333
3,Aguascalientes,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,0.000000
4,Aguascalientes,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0.000000


In [9]:
# Group murders per State per Year
murders = muni_crime_df[muni_crime_df['Tipo de delito'] == 'Homicidio']
murders_per_state = murders.groupby(['Entidad']).sum().sort_values(by='2022 AVG')
murders_per_state.sort_values('2022 AVG', ascending=False)

,2022 AVG
Entidad,
México,310.833333
Guanajuato,288.333333
Michoacán de Ocampo,265.333333
Baja California,248.000000
Jalisco,216.166667
Chihuahua,170.666667
Nuevo León,160.500000
Veracruz de Ignacio de la Llave,158.166667
Oaxaca,147.000000


In [10]:
# Count Unique Municipalities
muni_crime_df['Municipio'].value_counts()

Benito Juárez            686
Ocampo                   588
Emiliano Zapata          588
No Especificado          490
Morelos                  490
                        ... 
Penjamillo                98
Pátzcuaro                 98
Paracho                   98
Parácuaro                 98
Santa María de la Paz     98
Name: Municipio, Length: 2331, dtype: int64

In [11]:
# Drop Non Specified and Other Municipalities 
muni_crime_df.drop(muni_crime_df[muni_crime_df['Municipio'] == 'No Especificado'].index, inplace = True)
muni_crime_df.drop(muni_crime_df[muni_crime_df['Municipio'] == 'Otros Municipios'].index, inplace = True)

# Replace long names Cities/Municipalities with shorter names 
muni_crime_df['Municipio'].replace('Rincón Chamula San Pedro','Rincón Chamula', inplace=True)
muni_crime_df['Municipio'].replace('Batopilas de Manuel Gómez Morín','Batopilas', inplace=True)
muni_crime_df['Municipio'].replace('Jonacatepec de Leandro Valle','Jonacatepec', inplace=True)
muni_crime_df['Municipio'].replace('Heroica Villa Tezoatlán de Segura y Luna, Cuna de la Independencia de Oaxaca','Tezoatlán', inplace=True)
muni_crime_df['Municipio'].replace('San Juan Mixtepec Distrito 08','San Juan Mixtepec', inplace=True)
muni_crime_df['Municipio'].replace('San Juan Mixtepec Distrito 26','San Juan Mixtepec', inplace=True)
muni_crime_df['Municipio'].replace('San Pedro Mixtepec Distrito 22','San Pedro Mixtepec', inplace=True)
muni_crime_df['Municipio'].replace('San Pedro Mixtepec Distrito 26','San Pedro Mixtepec', inplace=True)

# Replace long names States/Entidades with shorter names 
muni_crime_df['Entidad'].replace('Veracruz de Ignacio de la Llave','Veracruz', inplace=True)
muni_crime_df['Entidad'].replace('Coahuila de Zaragoza','Coahuila', inplace=True)
muni_crime_df['Entidad'].replace('Michoacán de Ocampo','Michoacán', inplace=True)

In [12]:
# Create a Municipalities Table
munis = muni_crime_df.groupby(['Entidad','Municipio']).sum().round(decimals=2).reset_index()
munis

,Entidad,Municipio,2022 AVG
0,Aguascalientes,Aguascalientes,2501.33
1,Aguascalientes,Asientos,73.17
2,Aguascalientes,Calvillo,82.00
3,Aguascalientes,Cosío,38.67
4,Aguascalientes,El Llano,33.33
...,...,...,...
2464,Zacatecas,Villa González Ortega,4.00
2465,Zacatecas,Villa Hidalgo,5.67
2466,Zacatecas,Villa de Cos,21.17
2467,Zacatecas,Villanueva,37.83


In [13]:
# Duplicated Cities/Municipios Names
munis[munis.Municipio.duplicated()]

,Entidad,Municipio,2022 AVG
161,Chihuahua,Aldama,40.00
196,Chihuahua,Juárez,2965.50
231,Ciudad de México,Cuauhtémoc,3023.67
240,Ciudad de México,Venustiano Carranza,1090.83
245,Coahuila,Allende,51.83
...,...,...,...
2434,Zacatecas,Loreto,34.83
2437,Zacatecas,Melchor Ocampo,0.67
2442,Zacatecas,Morelos,22.17
2448,Zacatecas,Pánuco,23.17


In [14]:
# Reduce dataset to Cities/Municipios with average of more than 50 crimes
munis = munis[munis['2022 AVG']>50].reset_index(drop=True)
print(len(munis))
munis.head()

412


,Entidad,Municipio,2022 AVG
0,Aguascalientes,Aguascalientes,2501.33
1,Aguascalientes,Asientos,73.17
2,Aguascalientes,Calvillo,82.00
3,Aguascalientes,Jesús María,275.00
4,Aguascalientes,Pabellón de Arteaga,95.83


In [15]:
# Geo locate Municipalities
geolocator = Nominatim(user_agent="mexico_explorer")
Lat = []
Lon = []

for i in range(len(munis)):
  location = geolocator.geocode(munis['Municipio'][i]+', '+munis['Entidad'][i])
  print('The geograpical coordinate of {} are {}, {}.'.format(i, latitude, longitude))
  Lat.append(location.latitude)
  Lon.append(location.longitude)

munis['Latitude'] = Lat
munis['Longitude'] = Lon

In [18]:
# create map of Mexico using latitude and longitude values
map_mexico = folium.Map(location=[25, -102], zoom_start=6)

# add markers to map
for lat, lon, State, Municipality in zip(munis['Latitude'], munis['Longitude'], munis['Entidad'], munis['Municipio']):
    label = '{}, {}'.format(State, Municipality)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=0.1,
        popup=label,
        color='red',
        fill=False,
        parse_html=False).add_to(map_mexico)  

heat_data = list(zip(munis['Latitude'], munis['Longitude'],munis['2022 AVG']))
plugins.HeatMap(heat_data, radius=25).add_to(map_mexico)
 
map_mexico

In [17]:
# Export Crime and location data back to Google Drive
muni_crime_df.to_csv('Municipal_Crime_2022.csv')
!cp Municipal_Crime_2022.csv "/content/drive/My Drive/Colab_Data/"
munis.to_csv('Municipalities_Location_2022.csv')
!cp Municipalities_Location_2022.csv "/content/drive/My Drive/Colab_Data/"